# Treasury Curve Report

This lessons works through how we can create our first report, the treasury curve report. It will take some of our functions created from before and translate it into an automated pipeline for writing an excel sheet for our report.

## Creating our First Workbook

Before we get to the stage of writing our treasury curve report, we need to make our first excel workbook using the openpyxl library. If you do not have openpyxl installed you can install it through pip install or however you prefer to install your python libraries.

In [1]:
! pip install openpyxl

With openpyxl, you can create an excel workbook to modify. To build a blank template, just call openpyxl.Workbook() and it will return a workbook object.

In [2]:
import openpyxl
wb = openpyxl.Workbook()

With our workbook, we can grab the active sheet through wb.active. This will return to us what the current active sheet is in the excel workbook.

In [3]:
# Grab the active worksheet
ws = wb.active

One way to index into cells of a worksheet is to give the letter and number combination and then set a value. So, for example, below we can see how to set the value for the A1 cell to 42.

In [4]:
# Write data to cell by the letter number combination
ws['A1'] = 42

The other way that we can index into cells is by passing the argument of row and column.

In [5]:
# Write data to cell by the row/column combination
ws.cell(row=1,column=5).value = 5
ws.cell(row=5,column=1).value = 5

Just in case the folder does not yet exist for the workbooks folder. We can create it using the os library.

In [6]:
# Save the file
import os

if not os.path.exists("Workbooks"):
    os.mkdir("Workbooks")

Then we can save the workbook with wb.save with a filename.

In [7]:
wb.save("Workbooks/Example.xlsx")

Now that we know how to create and modify a workbook we can read in our data to use it.

In [8]:
import pandas as pd

# Read in table
yield_curve_table = pd.read_csv("Tables/Treasury Yield Table.csv", index_col=0)
print(yield_curve_table)

       Current  1 Year Ago Curve  1 Year Change  5 Years Ago Curve  \
1 Mo      1.48              2.44          -0.96               0.03   
3 Mo      1.55              2.45          -0.90               0.04   
6 Mo      1.60              2.56          -0.96               0.12   
1 Yr      1.59              2.63          -1.04               0.25   
2 Yr      1.58              2.48          -0.90               0.67   
3 Yr      1.62              2.46          -0.84               1.10   
5 Yr      1.69              2.51          -0.82               1.65   
7 Yr      1.83              2.59          -0.76               1.97   
10 Yr     1.92              2.69          -0.77               2.17   
20 Yr     2.25              2.87          -0.62               2.47   
30 Yr     2.39              3.02          -0.63               2.75   

       5 Year Change  
1 Mo            1.45  
3 Mo            1.51  
6 Mo            1.48  
1 Yr            1.34  
2 Yr            0.91  
3 Yr            0.52 

## Creating our Table Writing Function

In case you have not seen it before, calling values with a DataFrame will return a numpy array of data and calling tolist() converts it to a nested list.

In [9]:
# Doing values.tolist() on a dataframe returns the following
print(yield_curve_table.reset_index().values.tolist())

[['1 Mo', 1.48, 2.44, -0.96, 0.03, 1.45], ['3 Mo', 1.55, 2.45, -0.9000000000000001, 0.04, 1.51], ['6 Mo', 1.6, 2.56, -0.96, 0.12, 1.48], ['1 Yr', 1.59, 2.63, -1.0399999999999998, 0.25, 1.34], ['2 Yr', 1.58, 2.48, -0.8999999999999999, 0.67, 0.91], ['3 Yr', 1.62, 2.46, -0.8399999999999999, 1.1, 0.52], ['5 Yr', 1.69, 2.51, -0.8199999999999998, 1.65, 0.04], ['7 Yr', 1.83, 2.59, -0.7599999999999998, 1.97, -0.1399999999999999], ['10 Yr', 1.92, 2.69, -0.77, 2.17, -0.25], ['20 Yr', 2.25, 2.87, -0.6200000000000001, 2.47, -0.2200000000000002], ['30 Yr', 2.39, 3.02, -0.6299999999999999, 2.75, -0.3599999999999999]]


Now that we know how to get the values out, we have everything we need to be able to create the function for writing the tables into the excel worksheet.

In [10]:
from openpyxl.worksheet.worksheet import Worksheet
from pandas import DataFrame

# Define a function to write a pandas dataframe to excel

def write_to_excel(ws: Worksheet, start_row: int,
                   start_column: int, df: DataFrame,
                   index: bool = True, columns: bool = True) -> None:
    """
    Function which takes a worksheet and writes a table of data to it.

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    start_row : int
        The row to begin on
    start_column : int
        The column to start on
    df : DataFrame
        The data to write
    index : bool, optional
        Whether to write the index labels. The default is True.
    columns : bool, optional
        Whether to write the column labels. The default is True.

    """
    
    # If we want to include the index, reset the index on the dataframe first
    if index:
        write_data = df.reset_index().values.tolist()
    else:
        write_data = df.values.tolist()
    
    # If we want to include columns we need to add the columns as the first in the nested lists
    if columns:
        # If we already added an index, then we are going to want also include a blank space since
        # we don't want a label above the index
        if index:
            write_data = [[""]+list(df.columns)]+write_data
        else:
            write_data = [list(df.columns)]+write_data
    
    # For every value iterate through finding the location to place it in
    for row in range(len(write_data)):
        for column in range(len(write_data[0])):
            ws.cell(row=start_row+row,column=start_column+column).value = write_data[row][column]

wb = openpyxl.Workbook()
ws = wb.active

#Examples of writing to excel
write_to_excel(ws, 1, 1, yield_curve_table)
write_to_excel(ws, 16, 1, yield_curve_table,index=False)
write_to_excel(ws, 31, 1, yield_curve_table,columns=False)
write_to_excel(ws, 46, 1, yield_curve_table,index=False,columns=False)

wb.save("Workbooks/Example 2.xlsx")

### Drawing Colors

We can change the colors in the workbook by using the attribute fill. We just need to get create a PatternFill object with the color we want. 

In [11]:
from openpyxl.styles import PatternFill

# Create a workbook
wb = openpyxl.Workbook()
ws = wb.active

# Write the table in
write_to_excel(ws, 3, 1, yield_curve_table)

# PatternFill is a way to get colors, here we pass the start and end color as the same and use a fill type solid
# The color used here is red
my_fill_color = PatternFill(start_color='FF0000',
                   end_color='FF0000',
                   fill_type='solid')

# The fill attribute of a cell is the color to use
ws.cell(row=1,column=1).fill = my_fill_color

wb.save("Workbooks/Example 3.xlsx")

### Red-Green Map

We want a function that takes a set of data and colors it red or green depending on if the value is negative or positive. For changes in the yield curve, this will make it much more readable to see the changes overall.

In [12]:
#Create a function to map red and green colors based on whether a field is positive or negative
def red_green_map(ws: Worksheet, df: DataFrame, start_row: int, start_column: int) -> None:
    """
    Function which colors cells green or red depending on if
    they are positive or negative

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    df : DataFrame
        The dataframe of data
    start_row : int
        The row to start coloring on
    start_column : int
        The color to start coloring on

    """
    
    # Get the values
    write_data = df.values.tolist()
    
    # Iterate over the rows and columns
    for row in range(len(write_data)):
        for column in range(len(write_data[0])):
            # Set the fill color to green for positive numbers
            if write_data[row][column] > 0:
                fill = fill = PatternFill(start_color='008000',
                   end_color='008000',
                   fill_type='solid')
            # Set the fill color to red for negative numbers
            elif write_data[row][column] < 0:
                fill = PatternFill(start_color='FF0000',
                   end_color='FF0000',
                   fill_type='solid')
                
            # Change the color for the cell
            ws.cell(row=start_row+row,column=start_column+column).fill = fill

# Create the excel workbook
wb = openpyxl.Workbook()
ws = wb.active

#Write the data to excel
write_to_excel(ws, 3, 1, yield_curve_table)

# We'll deal with only the two change columns
# So pass the data for only those and also pass their starting columns/rows
red_green_map(ws, yield_curve_table[["1 Year Change"]], 4, 4)
red_green_map(ws, yield_curve_table[["5 Year Change"]], 4, 6)

wb.save("Workbooks/Example 4.xlsx")

We finished some of the basic functionality we are going to want to use. Now we can write the function to wrap these functions up that should be used to fill out the treasury sheet.

In [13]:
#Put it all together for a function that takes a worksheet and yield curve data then writes the sheet
def write_treasury_sheet(ws: Worksheet, yield_curve_table: DataFrame):
    """
    Function to fill out the treasury curve data

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    yield_curve_table : DataFrame
        The yield curve data


    """
    # Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    # Color the yield curve changes red/green
    red_green_map(ws, yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(ws, yield_curve_table[["5 Year Change"]], 4, 6)
    
wb = openpyxl.Workbook()
ws = wb.active
write_treasury_sheet(ws, yield_curve_table)
wb.save("Workbooks/Economic Report.xlsx")

## More Modifications for Our Excel Worksheet

### Converting Column Number to Letter

You may have noticed that in excel, the columns are letters. While we can use the number for the column most of the time, we might at times need the letter version. To convert an integer version of the column to the letter version, you can use get_column_letter.

In [14]:
from openpyxl.utils.cell import get_column_letter

# The get_column_letter function will convert a number to a letter for excel
print(get_column_letter(1))

A


### Adjusting Column Width

The way to adjust the width of a column in excel is to use ws.column_dimensions and then change the width of column (which is found by indexing into ws.column_dimensions with the letter version of the column).


In [15]:
from typing import List

def adjust_column_widths(ws: Worksheet, start: int, values: List[int]):
    """
    Function to adjust the width of the columns starting at the start value
    to the values listed in values

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    start : int
        The column to begin adjustments at
    values : List[int]
        The widths to set each column to in order


    """
    # Iterate through the values
    for i, width in enumerate(values):
        # Set the width of the column
        ws.column_dimensions[get_column_letter(start+i)].width = width

wb = openpyxl.Workbook()
ws = wb.active

write_to_excel(ws, 3, 1, yield_curve_table)

adjust_column_widths(ws,1,[2,10,20,5])

wb.save("Workbooks/Example 5.xlsx")

Given our new function, we want to update the write_treasury_sheet function. We will update the column lengths starting at column 2 (B) to have a length of 15.

In [16]:
# Update the function

def write_treasury_sheet(ws: Worksheet, yield_curve_table: DataFrame) -> None:
    """
    Function to fill out the treasury curve data

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    yield_curve_table : DataFrame
        The yield curve data

    """
    #Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    #Color the yield curve changes red/green
    red_green_map(ws, yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(ws, yield_curve_table[["5 Year Change"]], 4, 6)
    
    #Adjust the column widths for the table
    adjust_column_widths(ws,2,[15]*len(yield_curve_table.columns))
    
wb = openpyxl.Workbook()
ws = wb.active
write_treasury_sheet(ws, yield_curve_table)
wb.save("Workbooks/Economic Report.xlsx")

### Font

For each cell, there is an attribute of font which can be modified to change the style, size, etc of the font. We access it through the font attribute of the cell, and we can change it by passing a Font object. Some of the things that can be modified within the font object is bolding with bold = True/False.

One thing to note is that fonts are immutable when taken from the attribute so you need to use copy if you want to take a font and modify it.

In [17]:
from openpyxl.styles import Font
from copy import copy

# Create a new workbook
wb = openpyxl.Workbook()
ws = wb.active

# Write our table like before
write_to_excel(ws, 3, 1, yield_curve_table)

# Create a font with bold = True
font = Font(bold=True)

# Set the font on the cell at A4 to the font we just defined
ws.cell(row=4,column=1).font = font

# Or we can take the current fond and modify it like so to make it bold
font = copy(ws.cell(row=3,column=2).font)
print(font)
font.bold = True
ws.cell(row=3,column=2).font = font

wb.save("Workbooks/Example 6.xlsx")

<openpyxl.styles.fonts.Font object>
Parameters:
name='Calibri', charset=None, family=2.0, b=False, i=False, strike=None, outline=None, shadow=None, condense=None, color=<openpyxl.styles.colors.Color object>
Parameters:
rgb=None, indexed=None, auto=None, theme=1, tint=0.0, type='theme', extend=None, sz=11.0, u=None, vertAlign=None, scheme='minor'


Now we want to make a function which goes through cells in a line and bolds them to make it easier to bold a column or row.

In [18]:
# Create the function

def bold_cells_line(ws: Worksheet, start_row: int, start_column: int,
                    length: int, vertical: bool = True) -> None:
    """
    Function which bolds a column or row of a worksheet

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    start_row : int
        The row to start at for bolding
    start_column : int
        The column to start at for bolding
    length : int
        The length of the column or row to bold through
    vertical : bool, optional
        Decides if it is a row or column of the parameter length
        that is bolded. The default is True.

    """
    
    # Iterate through the length of the row/column
    for i in range(length):
        # Increment the row or column depending on if vertical is true
        if vertical:
            cell = ws.cell(row=start_row+i,column=start_column)
        else:
            cell = ws.cell(row=start_row,column=start_column+i)
            
        # Copy the font and set the bold property to true
        font = copy(cell.font)
        font.bold = True
        cell.font = font
        
# Update the treasury sheet function
def write_treasury_sheet(ws: Worksheet, yield_curve_table: DataFrame) -> None:
    """
    Function to fill out the treasury curve data

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    yield_curve_table : DataFrame
        The yield curve data

    """
    #Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    #Color the yield curve changes red/green
    red_green_map(ws, yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(ws, yield_curve_table[["5 Year Change"]], 4, 6)
    
    #Adjust the column widths for the table
    adjust_column_widths(ws,2,[15]*len(yield_curve_table.columns))
    
    #Bold the index and columns
    bold_cells_line(ws, 4, 1, len(yield_curve_table), vertical=True)
    bold_cells_line(ws, 3, 2, len(yield_curve_table.columns), vertical=False)
    
    
# Test the new function
wb = openpyxl.Workbook()
ws = wb.active
write_treasury_sheet(ws, yield_curve_table)
wb.save("Workbooks/Economic Report.xlsx")

### Merging Cells

We can merge cells together by passing a range in the format X:Y where X is the upper left corner and Y is the bottom right corner. For example A1:F1 passed into the merge_cells function will merge those cells together. Below is an example of it.

In [19]:
# Set up the worksheet
wb = openpyxl.Workbook()
ws = wb.active
write_to_excel(ws, 3, 1, yield_curve_table)

# Merging cells can be done with ws.merge_cells
ws.merge_cells('A1:F1')

wb.save("Workbooks/Example 7.xlsx")

### Getting Coordinates

There are two more functions we want to learn more about as they will help us when we move on to some other improvements in our workbook.

1. coordinate_from_string: This returns the coordinates for the string representation of a cell  
2. column_index_from_string: This converts a letter to the actual number in the excel sheet for a coordinate

Below we show both examples.

In [20]:
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string

# coordinate_from_string returns the coordinates for the string representation of a cell  
print(coordinate_from_string("B6"))

# column_index_from_string converts a letter to the actual number in the excel sheet for a coordinate
print(column_index_from_string("B"))

('B', 6)
2


### Title Function Creation Challenge

Given what we have learned so far, here is a challenge. Create a function for creating the title in a workbook which takes as arguments the worksheet, the area to make the title (such as A1:A5), and the text to put in the title.

It should do the following:
1. Merge the cells for the area
2. Find the row/column of the most upper left cell in the area parameter
3. Set the text of this cell to the text (which will set it for the full merged cell)
4. Change the font of this cell to be of size 16, with bold as true, and a color of "283747".

We will also learn how to do alignment when we see the answer!

## Borders and Images

In this section, we build on how to create borders and images after we work through the title function creation solution.

### Title Function Creation Solution

Below is the solution to the problem. As well, we can see the last line sets the alignment to horizontal by creating an Alignment object.

In [21]:
from openpyxl.styles.alignment import Alignment

# A title function for the worksheet
def create_title(ws: Worksheet, area: str, text: str) -> None:
    """
    Writes a title on to the passed worksheet

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    area : str
        The area to put the title in (of the form A1:A5 or similar)
    text : str
        The text for the title

    """
    # Merge the cells in the area
    ws.merge_cells(area)
    
    # Given an area such as A1:A5, we want to first isolate the first cell
    # Split will break the area into the two components, we take the first one
    first_cell = area.split(":")[0]
    
    # Find the coordinates
    first_cell = coordinate_from_string(first_cell)
    
    # Convert the components to the numerical values for column and row
    col = column_index_from_string(first_cell[0])
    row = first_cell[1]
    
    # Change the text of the merged cell to whatever was passed as input
    ws.cell(row=row, column=col).value = text
    
    # Change the font of the title
    font = font = Font(size=16,
                       bold=True,
                      color="283747")
    ws.cell(row=row, column=col).font = font
    
    # Change alignment to be centered
    ws.cell(row=row, column=col).alignment = Alignment(horizontal="center")


# Update the treasury sheet function
def write_treasury_sheet(ws: Worksheet, yield_curve_table: DataFrame) -> None:
    """
    Function to fill out the treasury curve data

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    yield_curve_table : DataFrame
        The yield curve data

    """
    #Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    #Color the yield curve changes red/green
    red_green_map(ws, yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(ws, yield_curve_table[["5 Year Change"]], 4, 6)
    
    #Adjust the column widths for the table
    adjust_column_widths(ws,2,[15]*len(yield_curve_table.columns))
    
    #Bold the index and columns
    bold_cells_line(ws, 4, 1, len(yield_curve_table), vertical=True)
    bold_cells_line(ws, 3, 2, len(yield_curve_table.columns), vertical=False)
    
    # Create title
    create_title(ws, 'A1:F1', "Treasury Yield Curve")
    
wb = openpyxl.Workbook()
ws = wb.active
write_treasury_sheet(ws, yield_curve_table)
wb.save("Workbooks/Economic Report.xlsx")

### Borders

For borders, a cell can be modified by using its border attribute.

What needs to be passed is a Border object in which we specify the borders for each side left, right, top and bottom. These borders can be any of the permissible excel borders. 

Below is an example of creating borders for a cell.

In [22]:
from openpyxl.styles import Border, Side

# Create the base workbook
wb = openpyxl.Workbook()
ws = wb.active
write_to_excel(ws, 3, 1, yield_curve_table)

# With borders we need to specify the styles we want to use for each border
border = Border(left=Side(border_style='thin'),
         right=Side(border_style='thin'),
         top=Side(border_style='thin'),
         bottom=Side(border_style='thin'))

# Set the border
temp = ws.cell(row=5,column=5)
temp.border = border

wb.save("Workbooks/Example 8.xlsx")

Now that we know how borders work, how about we build a function for giving thin borders in a region of our worksheet.

In [23]:
#Create a function which takes a starting position and the length vertically and horizontally
#and then creates borders around all of them
def create_border(ws: Worksheet, start_row: int, start_column: int,
                  vertical_length: int, horizontal_length: int) -> None:
    """
    Function which takes a starting position and the length vertically
    and horizontally, then creates thin borders around all of them

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    start_row : int
        The row to begin at
    start_column : int
        The column to begin at
    vertical_length : int
        How far down borders should be applied
    horizontal_length : int
        How far right borders should be applied
    """
    # Create the border
    border = Border(left=Side(border_style='thin'),
         right=Side(border_style='thin'),
         top=Side(border_style='thin'),
         bottom=Side(border_style='thin'))
    
    # Iterate and set the border
    for row in range(vertical_length):
        for column in range(horizontal_length):
            ws.cell(row=start_row+row,column=start_column+column).border = border
            
# Create the base notebook
wb = openpyxl.Workbook()
ws = wb.active
write_to_excel(ws, 3, 1, yield_curve_table)

# Write the border
create_border(ws, 4, 2, yield_curve_table.shape[0], yield_curve_table.shape[1])

wb.save("Workbooks/Example 9.xlsx")

Naturally, we can update our main function for writing at this point.

In [24]:
def write_treasury_sheet(ws: Worksheet, yield_curve_table: DataFrame) -> None:
    """
    Function to fill out the treasury curve data

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    yield_curve_table : DataFrame
        The yield curve data

    """
    # Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    # Color the yield curve changes red/green
    red_green_map(ws, yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(ws, yield_curve_table[["5 Year Change"]], 4, 6)
    
    # Adjust the column widths for the table
    adjust_column_widths(ws,2,[15]*len(yield_curve_table.columns))
    
    # Bold the index and columns
    bold_cells_line(ws, 4, 1, len(yield_curve_table), vertical=True)
    bold_cells_line(ws, 3, 2, len(yield_curve_table.columns), vertical=False)
    
    # Create title
    create_title(ws, 'A1:F1', "Treasury Yield Curve")
    
    # Add borders
    create_border(ws, 4, 2, yield_curve_table.shape[0], yield_curve_table.shape[1])
    
wb = openpyxl.Workbook()
ws = wb.active
write_treasury_sheet(ws, yield_curve_table)
wb.save("Workbooks/Economic Report.xlsx")

### Gridlines

For the excel worksheet, there is an attribute sheet_view which has a few other modifications we can make. One of those is the attribute showGridLines which when set to false will get rid of the gridlines in the worksheet. Below is an example of turning it off.

In [25]:
# Write the base worksheet
wb = openpyxl.Workbook()
ws = wb.active
write_to_excel(ws, 3, 1, yield_curve_table)

# We might be interested in switching off gridlines
ws.sheet_view.showGridLines = False

wb.save("Workbooks/Example 10.xlsx")

Then update the write_treasury_sheet function.

In [26]:
def write_treasury_sheet(ws: Worksheet, yield_curve_table: DataFrame) -> None:
    """
    Function to fill out the treasury curve data

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    yield_curve_table : DataFrame
        The yield curve data

    """
    # Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    # Color the yield curve changes red/green
    red_green_map(ws, yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(ws, yield_curve_table[["5 Year Change"]], 4, 6)
    
    # Adjust the column widths for the table
    adjust_column_widths(ws,2,[15]*len(yield_curve_table.columns))
    
    # Bold the index and columns
    bold_cells_line(ws, 4, 1, len(yield_curve_table), vertical=True)
    bold_cells_line(ws, 3, 2, len(yield_curve_table.columns), vertical=False)
    
    # Create title
    create_title(ws, 'A1:F1', "Treasury Yield Curve")
    
    # Add borders
    create_border(ws, 4, 2, yield_curve_table.shape[0], yield_curve_table.shape[1])
    
    # Get rid of gridlines
    ws.sheet_view.showGridLines = False
    
wb = openpyxl.Workbook()
ws = wb.active
write_treasury_sheet(ws, yield_curve_table)
wb.save("Workbooks/Economic Report.xlsx")

### Adding Images

With the function add_image, we just need to specify and image and where to place it and we will then have it placed nicely in our worksheet. For example, below, we pull in the treasury yield curve image we created in the last lesson as an image that we want to add to our workbook. Then we call add image and give it the location of H1 that we want it placed at.

In [27]:
from openpyxl.drawing.image import Image

# Create the base worksheet
wb = openpyxl.Workbook()
ws = wb.active
write_to_excel(ws, 3, 1, yield_curve_table)

# Images can be put into excel worksheets by loading them then adding image to a given position
img = Image('Images/Treasury Yield Curves.png')
ws.add_image(img, 'H1')

wb.save("Workbooks/Example 11.xlsx")

Add the final modifications to our write_treasury_sheet function.

In [28]:
def write_treasury_sheet(ws: Worksheet, yield_curve_table: DataFrame) -> None:
    """
    Function to fill out the treasury curve data

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    yield_curve_table : DataFrame
        The yield curve data

    """
    # Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    # Color the yield curve changes red/green
    red_green_map(ws, yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(ws, yield_curve_table[["5 Year Change"]], 4, 6)
    
    # Adjust the column widths for the table
    adjust_column_widths(ws,2,[15]*len(yield_curve_table.columns))
    
    # Bold the index and columns
    bold_cells_line(ws, 4, 1, len(yield_curve_table), vertical=True)
    bold_cells_line(ws, 3, 2, len(yield_curve_table.columns), vertical=False)
    
    # Create title
    create_title(ws, 'A1:F1', "Treasury Yield Curve")
    
    # Add borders
    create_border(ws, 4, 2, yield_curve_table.shape[0], yield_curve_table.shape[1])
    
    # Get rid of gridlines
    ws.sheet_view.showGridLines = False
    
    # Add yield curve graphs
    img = Image('Images/Treasury Yield Curves.png')
    ws.add_image(img, 'H1')
    
wb = openpyxl.Workbook()
ws = wb.active
write_treasury_sheet(ws, yield_curve_table)
wb.save("Workbooks/Economic Report.xlsx")